<a id="TableOfContents"></a>
# TABLE OF CONTENTS:
<li><a href='#imports'>Imports</a></li>
<li><a href="#telco">Get Telco Dataset</a></li>
<li><a href="#split">Split Telco Dataset</a></li>
<li><a href="#keycol">Key Columns</a></li>
<li><a href="#DTC">Decision Tree Classifier Modeling</a></li>
<li><a href="#RFC">Random Forest Classifier Modeling</a></li>
<li><a href="#KNN">K-Nearest Neighbors Modeling</a></li>
<li><a href="#LR">Logistic Regression Modeling</a></li>
<li><a href="#top3">Top 3 Models</a></li>

##### Orientation:
The purpose of this file is to create models to predict whether or not a customer will churn and get the best performing models.

<a id='imports'></a>
# IMPORTS:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [1]:
# tabular data
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# DecisionTree modeling:
from sklearn.tree import DecisionTreeClassifier as DTC, export_text, plot_tree

# RandomForest modeling:
from sklearn.ensemble import RandomForestClassifier as RFC

# KNN modeling:
from sklearn.neighbors import KNeighborsClassifier as KNN

# Logistic Regression modeling:
from sklearn.linear_model import LogisticRegression as LR

# Other sklearn stuff
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# .py files
import prepare
import evaluation
import model

from sklearn.feature_selection import SelectKBest, chi2

<a id='telco'></a>
# Get Telco Dataset:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [2]:
# Acquire cleaned 'telco' dataset
telco = prepare.prep_telco()
telco.sample()

/usr/local/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,sign_dayofweek_1,sign_dayofweek_2,sign_dayofweek_3,sign_dayofweek_4,sign_dayofweek_5,sign_dayofweek_6,total_services,total_extra_services,value_per_total_services,value_per_total_extra_services
5432,Female,0,Yes,Yes,63,Yes,No,No internet service,No internet service,No internet service,...,0,0,0,1,0,0,1,0,19.35,inf


<a id='split'></a>
# Split Telco Dataset:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [3]:
# Split telco dataset
train, val, test = prepare.split(telco, 'churn')

train.shape:(3943, 100)
validate.shape:(1691, 100)
test.shape:(1409, 100)


In [4]:
# Verify split shapes
train.shape, val.shape, test.shape

((3943, 100), (1691, 100), (1409, 100))

<a id='keycol'></a>
# Key Columns:
<li><a href='#TableOfContents'>Table of Contents</a></li>

- Columns:
    - 'partner'
    - 'dependents'
    - 'online_security'
    - 'online_backup'
    - 'device_protection'
    - 'tech_support'
    - 'streaming_tv'
    - 'streaming_movies'
    - 'payment_type'
    - 'contract_type'
    - 'total_services'
    - 'total_extra_services'

# List of pertinent columns
keylist = [
    'partner_Yes',
    'partner_No',
    'dependents_Yes',
    'dependents_No',
    'online_security_Yes',
    'online_security_No',
    'online_security_No_internet_service',
    'online_backup_Yes',
    'online_backup_No',
    'online_backup_No_internet_service',
    'device_protection_Yes',
    'device_protection_No',
    'device_protection_No_internet_service',
    'tech_support_Yes',
    'tech_support_No',
    'tech_support_No_internet_service',
    'streaming_tv_Yes',
    'streaming_tv_No',
    'streaming_tv_No_internet_service',
    'streaming_movies_Yes',
    'streaming_movies_No',
    'streaming_movies_No_internet_service',
    'payment_type_Bank_transfer_(automatic)',
    'payment_type_Credit_card_(automatic)',
    'payment_type_Electronic_check',
    'payment_type_Mailed_check',
    'contract_type_Month-to-month',
    'contract_type_One_year',
    'contract_type_Two_year',
    'total_services',
    'total_extra_services'
]

In [228]:
# List of pertinent columns
# Bin signup_date
keylist = [
    'partner_No',
    'dependents_No',
    'online_security_No',
    'online_backup_No',
    'device_protection_No',
    'tech_support_No',
    'paperless_billing_Yes',
    'contract_type_Month-to-month',
    'internet_service_type_Fiber_optic',
    'payment_type_Electronic_check',
    'sign_year',
    'sign_month',
    'sign_dayofweek',
    'total_services',
    'total_extra_services',
    'senior_citizen',
    'tenure',
    'monthly_charges',
    'value_per_total_services'
]

In [229]:
# Assign x/y train/val/test cols:
x_train = train[keylist]
y_train = train['churn']
x_val = val[keylist]
y_val = val['churn']
x_test = test[keylist]
y_test = test['churn']

<a id='DTC'></a>
# Decision Tree Classifier Modeling:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [230]:
# Create base dictionary
dtcscores = {
    'model' : ['actual'],
    'train' : [100],
    'val' : [100],
    'diff' : [0],
    'test' : [100]
}

In [231]:
# Add baseline to dictionary
baselinetrain = round((train.churn == 'No').mean(), 5)
baselineval = round((val.churn == 'No').mean(), 5)
baselinediff = round(abs(baselinetrain - baselineval), 5)
baselinetest = round((test.churn == 'No').mean(), 5)
dtcscores['model'].append('baseline')
dtcscores['train'].append(baselinetrain)
dtcscores['val'].append(baselineval)
dtcscores['diff'].append(baselinediff)
dtcscores['test'].append(baselinetest)

In [232]:
# Confirm df funcitonality with actual and baseline
pd.DataFrame.from_dict(dtcscores)

,model,train,val,diff,test
0,actual,100.00000,100.00000,0.00000,100.00000
1,baseline,0.73472,0.73448,0.00024,0.73456


In [233]:
# Make and fit DTC of 2-15 depth to train
for i in range(2, 16):
    dtc = DTC(max_depth=i, random_state=100)
    dtc.fit(x_train, y_train)
    model = dtc.predict(x_train)
    trainscore = round(dtc.score(x_train, y_train), 5)
    valscore = round(dtc.score(x_val, y_val), 5)
    diffscore = round(abs(trainscore - valscore), 5)
    testscore = round(dtc.score(x_test, y_test), 5)
    dtcscores['model'].append(f'model{i}')
    dtcscores['train'].append(trainscore)
    dtcscores['val'].append(valscore)
    dtcscores['diff'].append(diffscore)
    dtcscores['test'].append(testscore)

TypeError: float() argument must be a string or a number, not 'NaTType'

In [225]:
# Find best 'train' model
pd.DataFrame.from_dict(dtcscores).sort_values(by='train', ascending=False).head(2)

,model,train,val,diff,test
0,actual,100.00000,100.0000,0.00000,100.00000
15,model15,0.97286,0.7191,0.25376,0.71398


In [226]:
# Find best 'val' model
pd.DataFrame.from_dict(dtcscores).sort_values(by='val', ascending=False).head(2)

,model,train,val,diff,test
0,actual,100.00000,100.00000,0.00000,100.00000
3,model3,0.79787,0.78001,0.01786,0.76579


In [227]:
# Find best 'test' model
pd.DataFrame.from_dict(dtcscores).sort_values(by='test', ascending=False).head(2)

,model,train,val,diff,test
0,actual,100.00000,100.00000,0.0000,100.0000
4,model4,0.80142,0.77292,0.0285,0.7665


In [142]:
dtc = DTC(max_depth=3, 
          min_impurity_decrease=0.006,
          random_state=100)
dtc.fit(x_train, y_train)
model = dtc.predict(x_test)
matrix = confusion_matrix(y_test, model, labels=('Yes', 'No'))
print(matrix)
(matrix[0, 0] / (matrix[0, 0] + matrix[1, 0]))

[[125 249]
 [ 78 957]]


0.6157635467980296

- Best Train Model:
    - model 15 @ 88.1%
        - Restrictions:
            - max_depth = 15
            - random_state=100
- Best Validate Model:
    - model 4 @ 76.8%
        - Restrictions:
            - max_depth = 4
            - random_state=100
- Best Test Model:
    - model 4 @ 76.0%
        - Restrictions:
            - max_depth = 4
            - random_state=100
- Baseline:
    - 73.5%

<a id='RFC'></a>
# Random Forest Classifier Modeling:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [ ]:
# Create base dictionary
rfcscores = {
    'model' : ['actual'],
    'train' : [100],
    'val' : [100],
    'diff' : [0],
    'test' : [100]
}

In [ ]:
# Add baseline to dictionary
baselinetrain = round((train.churn == 'No').mean(), 5)
baselineval = round((val.churn == 'No').mean(), 5)
baselinediff = round(abs(baselinetrain - baselineval), 5)
baselinetest = round((test.churn == 'No').mean(), 5)
rfcscores['model'].append('baseline')
rfcscores['train'].append(baselinetrain)
rfcscores['val'].append(baselineval)
rfcscores['diff'].append(baselinediff)
rfcscores['test'].append(baselinetest)

In [ ]:
# Confirm df funcitonality with actual and baseline
pd.DataFrame.from_dict(rfcscores)

In [ ]:
# Make and fit RFC of 2-15 depth to train
for i in range(2, 16):
    rfc = RFC(max_depth=i, random_state=100)
    rfc.fit(x_train, y_train)
    model = rfc.predict(x_train)
    trainscore = round(rfc.score(x_train, y_train), 5)
    valscore = round(rfc.score(x_val, y_val), 5)
    diffscore = round(abs(trainscore - valscore), 5)
    testscore = round(rfc.score(x_test, y_test), 5)
    rfcscores['model'].append(f'model{i}')
    rfcscores['train'].append(trainscore)
    rfcscores['val'].append(valscore)
    rfcscores['diff'].append(diffscore)
    rfcscores['test'].append(testscore)

In [ ]:
# Find best 'train' model
pd.DataFrame.from_dict(rfcscores).sort_values(by='train', ascending=False).head(2)

In [ ]:
# Find best 'val' model
pd.DataFrame.from_dict(rfcscores).sort_values(by='val', ascending=False).head(2)

In [ ]:
# Find best 'test' model
pd.DataFrame.from_dict(rfcscores).sort_values(by='test', ascending=False).head(2)

- Best Train Model:
    - model 15 @ 88.1%
        - Restrictions:
            - max_depth = 15
            - random_state=100
- Best Validate Model:
    - model 6 @ 76.6%
        - Restrictions:
            - max_depth = 6
            - random_state=100
- Best Test Model:
    - model 6 @ 76.7%
        - Restrictions:
            - max_depth = 6
            - random_state=100
- Baseline:
    - 73.5%

<a id='KNN'></a>
# K-Nearest Neighbors Modeling:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [ ]:
# Create base dictionary
knnscores = {
    'model' : ['actual'],
    'train' : [100],
    'val' : [100],
    'diff' : [0],
    'test' : [100]
}

In [ ]:
# Add baseline to dictionary
baselinetrain = round((train.churn == 'No').mean(), 5)
baselineval = round((val.churn == 'No').mean(), 5)
baselinediff = round(abs(baselinetrain - baselineval), 5)
baselinetest = round((test.churn == 'No').mean(), 5)
knnscores['model'].append('baseline')
knnscores['train'].append(baselinetrain)
knnscores['val'].append(baselineval)
knnscores['diff'].append(baselinediff)
knnscores['test'].append(baselinetest)

In [ ]:
# Confirm df funcitonality with actual and baseline
pd.DataFrame.from_dict(knnscores)

In [ ]:
# Make and fit KNN of 1-15 neighbors to train
for i in range(1, 16):
    knn = KNN(n_neighbors=i)
    knn.fit(x_train, y_train)
    model = knn.predict(x_train)
    trainscore = round(knn.score(x_train, y_train), 5)
    valscore = round(knn.score(x_val, y_val), 5)
    diffscore = round(abs(trainscore - valscore), 5)
    testscore = round(knn.score(x_test, y_test), 5)
    knnscores['model'].append(f'model{i}')
    knnscores['train'].append(trainscore)
    knnscores['val'].append(valscore)
    knnscores['diff'].append(diffscore)
    knnscores['test'].append(testscore)

In [ ]:
# Find best 'train' model
pd.DataFrame.from_dict(knnscores).sort_values(by='train', ascending=False).head(2)

In [ ]:
# Find best 'val' model
pd.DataFrame.from_dict(knnscores).sort_values(by='val', ascending=False).head(2)

In [ ]:
# Find best 'test' model
pd.DataFrame.from_dict(knnscores).sort_values(by='test', ascending=False).head(2)

- Best Train Model:
    - model 1 @ 84.8%
        - Restrictions:
            - n_neighbors = 1
- Best Validate Model:
    - model 12 @ 74.7%
        - Restrictions:
            - n_neighbors = 12
- Best Test Model:
    - model 14 @ 74.9%
        - Restrictions:
            - n_neighbors = 14
- Baseline:
    - 73.5%

<a id='LR'></a>
# Logistic Regression Modeling:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [ ]:
# Create base dictionary
lrscores = {
    'model' : ['actual'],
    'train' : [100],
    'val' : [100],
    'diff' : [0],
    'test' : [100]
}

In [ ]:
# Add baseline to dictionary
baselinetrain = round((train.churn == 'No').mean(), 5)
baselineval = round((val.churn == 'No').mean(), 5)
baselinediff = round(abs(baselinetrain - baselineval), 5)
baselinetest = round((test.churn == 'No').mean(), 5)
lrscores['model'].append('baseline')
lrscores['train'].append(baselinetrain)
lrscores['val'].append(baselineval)
lrscores['diff'].append(baselinediff)
lrscores['test'].append(baselinetest)

In [ ]:
# Confirm df funcitonality with actual and baseline
pd.DataFrame.from_dict(lrscores)

In [ ]:
# Make and fit LR to train
lr = LR(random_state=100)
lr.fit(x_train, y_train)
model = lr.predict(x_train)
trainscore = round(lr.score(x_train, y_train), 5)
valscore = round(lr.score(x_val, y_val), 5)
diffscore = round(abs(trainscore - valscore), 5)
testscore = round(lr.score(x_test, y_test), 5)
lrscores['model'].append('model1')
lrscores['train'].append(trainscore)
lrscores['val'].append(valscore)
lrscores['diff'].append(diffscore)
lrscores['test'].append(testscore)

In [ ]:
# Find best 'train' model
pd.DataFrame.from_dict(lrscores).sort_values(by='train', ascending=False).head(2)

- Best Train Model:
    - model 1 @ 79.5%
        - Restrictions:
            - random_state=100
- Best Validate Model:
    - model 1 @ 75.9%
        - Restrictions:
            - random_state=100
- Best Test Model:
    - model 1 @ 76.2%
        - Restrictions:
            - random_state=100
- Baseline:
    - 73.5%

<a id='top3'></a>
# Top 3 Models:
<li><a href='#TableOfContents'>Table of Contents</a></li>